In [136]:
import numpy as np
import nltk
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize

In [2]:
# reading the input file to memory
df = pd.read_excel('Input.xlsx', sheet_name='Sheet1')

# Web Scraping 170 webpages using Beautifulsoup and requests modules to collect data


###  timing the run. just over 4 minutes. Maybe we can reduce runtime if we establish a single session instead of making requests iteratively.

In [27]:
import time
start = time.time()

titles = []
body_text = []

for i in df.URL:
    page_url = i
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    html_text = requests.get(page_url, headers = headers).text

    soup = bs(html_text, 'lxml')

    title = soup.find('h1', class_ = 'entry-title').text
    titles.append(title)

    body_div = soup.find('div', class_ = 'td-post-content').text    
    body_text.append(body_div)
    
end = time.time()
print(end - start)

253.24933338165283


## Merging the scraped text with input dataframe for further analysis.

In [34]:
x = pd.Series(titles)
y = pd.Series(body_text)
df1 = pd.DataFrame(x, columns = ['Title'])
df2 = pd.DataFrame(y, columns = ['Body'])
result = df.join(df1)
result = result.join(df2)

# Trying to dump extracted text to their respective URL_ID text files
##  enter your own specific path of local disk for correct generation of files.

In [122]:
for i in result.index:
    url = result['URL_ID'][i]
    # enter your own specific path of local disk for correct generation of files.
    path = r'Z:\Major Project\assignment\20211030 Test Assignment\New folder' + '\\' + str(url) + '.txt'
    title = result.Title[i]
    body = result.Body[i]
    with open(path ,'w', encoding='utf-8') as f:
        f.write(title)
        f.write('\n\n')
        f.writelines(body)
        f.close()

# dumping dataframe with Title and Body to a csv file in disk for future analysis and avoiding webscraping again.

In [15]:
result.to_csv('BlackCoffer.csv')

# Reading from the dumped csv above to avoid webscraping again.

In [3]:
result = pd.read_csv('BlackCoffer.csv')
result.drop(columns = ['Unnamed: 0'],inplace=True)
result = result.replace('\n',' ', regex=True)
result

# Reading stopwords and Master dictionary to memory for analysis

In [154]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\MasterDictionary\negative-words.txt', 'r') as f:
    lines = f.readlines()
    neg_words = []
    for l in lines:
        neg_words.append(l.replace("\n", ""))

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

In [155]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\MasterDictionary\positive-words.txt', 'r') as f:
    lines = f.readlines()
    pos_words = []
    for l in lines:
        pos_words.append(l.replace("\n", ""))

In [31]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_Auditor.txt', 'r') as f:
    lines = f.readlines()
    audit_stopword = []
    for l in lines:
        lst = l.split('|')
        audit_stopword.append(lst[0].replace("\n",""))

In [33]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_Currencies.txt', 'r') as f:
    lines = f.readlines()
    curr_stopword = []
    for l in lines:
        lst = l.split('|')
        curr_stopword.append(lst[0].replace("\n",""))
curr_stopword = list(map(lambda x: x.replace(" ",""), curr_stopword))

In [38]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_DatesandNumbers.txt', 'r') as f:
    lines = f.readlines()
    datenum_stopword = []
    for l in lines:
        lst = l.split('|')
        datenum_stopword.append(lst[0].replace("\n",""))
datenum_stopword = list(map(lambda x: x.replace(" ",""), datenum_stopword))

In [43]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_Generic.txt', 'r') as f:
    lines = f.readlines()
    generic_stopword = []
    for l in lines:
        generic_stopword.append(l.replace("\n",""))

['ABOUT',
 'ABOVE',
 'AFTER',
 'AGAIN',
 'ALL',
 'AM',
 'AMONG',
 'AN',
 'AND',
 'ANY',
 'ARE',
 'AS',
 'AT',
 'BE',
 'BECAUSE',
 'BEEN',
 'BEFORE',
 'BEING',
 'BELOW',
 'BETWEEN',
 'BOTH',
 'BUT',
 'BY',
 'CAN',
 'DID',
 'DO',
 'DOES',
 'DOING',
 'DOWN',
 'DURING',
 'EACH',
 'FEW',
 'FOR',
 'FROM',
 'FURTHER',
 'HAD',
 'HAS',
 'HAVE',
 'HAVING',
 'HE',
 'HER',
 'HERE',
 'HERS',
 'HERSELF',
 'HIM',
 'HIMSELF',
 'HIS',
 'HOW',
 'IF',
 'IN',
 'INTO',
 'IS',
 'IT',
 'ITS',
 'ITSELF',
 'JUST',
 'ME',
 'MORE',
 'MOST',
 'MY',
 'MYSELF',
 'NO',
 'NOR',
 'NOT',
 'NOW',
 'OF',
 'OFF',
 'ON',
 'ONCE',
 'ONLY',
 'OR',
 'OTHER',
 'OUR',
 'OURS',
 'OURSELVES',
 'OUT',
 'OVER',
 'OWN',
 'SAME',
 'SHE',
 'SHOULD',
 'SO',
 'SOME',
 'SUCH',
 'THAN',
 'THAT',
 'THE',
 'THEIR',
 'THEIRS',
 'THEM',
 'THEMSELVES',
 'THEN',
 'THERE',
 'THESE',
 'THEY',
 'THIS',
 'THOSE',
 'THROUGH',
 'TO',
 'TOO',
 'UNDER',
 'UNTIL',
 'UP',
 'VERY',
 'WAS',
 'WE',
 'WERE',
 'WHAT',
 'WHEN',
 'WHERE',
 'WHICH',
 'WHILE',
 '

In [44]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_GenericLong.txt', 'r') as f:
    lines = f.readlines()
    genlong_stopword = []
    for l in lines:
        genlong_stopword.append(l.replace("\n",""))

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

In [45]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_Geographic.txt', 'r') as f:
    lines = f.readlines()
    geo_stopword = []
    for l in lines:
        lst = l.split('|')
        geo_stopword.append(lst[0].replace("\n",""))
geo_stopword = list(map(lambda x: x.replace(" ",""), geo_stopword))

['UNITED',
 'STATE',
 'NORTH',
 'SOUTH',
 'EAST',
 'NORTHEAST',
 'NORTHWEST',
 'SOUTHEAST',
 'SOUTHWEST',
 'WEST',
 'OCEAN',
 'SEA',
 'LAKE',
 'RIVER',
 'CREEK',
 'GULF',
 'MOUNTAIN',
 'STREET',
 'BOULEVARD',
 'BLVD',
 'PARKWAY',
 'CITY',
 'COUNTY',
 'COUNTRY',
 'PACIFIC',
 'ATLANTIC',
 'INDIAN',
 'MEDITERRANEAN',
 'COMMONWEALTH',
 'AMERICA',
 'AMERICAN',
 'YORK',
 'CHICAGO',
 'LAS',
 'VEGAS',
 'LOS',
 'ANGELES',
 'MILWAUKEE',
 'SUNNYVALE',
 'FREMONT',
 'CINCINNATI',
 'PHILADELPHIA',
 'MIAMI',
 'DALLAS',
 'FORT',
 'BOSTON',
 'HOUSTON',
 'WASHINGTON',
 'ATLANTA',
 'DETROIT',
 'SAN',
 'FRANSICO',
 'PHOENIX',
 'SEATTLE',
 'DIEGO',
 'MINNEAPOLIS',
 'MEMPHIS',
 'DENVER',
 'ST',
 'LOUIS',
 'PITTSBURGH',
 'MANHATTAN',
 'HOLLYWOOD',
 'COLUMBUS',
 'INDIANAPOLIS',
 'MUMBAI',
 'KARACHI',
 'ONTARIO',
 'TORONTO',
 'CAMBRIDGE',
 'DELHI',
 'SAO',
 'PAULO',
 'SHANGHAI',
 'MOSCOW',
 'SEOUL',
 'ISTANBUL',
 'TOKYO',
 'JAKARTA',
 'BEIJING',
 'LONDON',
 'LUXEMBOURG',
 'SINGAPORE',
 'REPUBLIC',
 'CHINA',
 '

In [46]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\StopWords\StopWords_Names.txt', 'r') as f:
    lines = f.readlines()
    name_stopword = []
    for l in lines:
        lst = l.split('|')
        name_stopword.append(lst[0].replace("\n",""))
name_stopword = list(map(lambda x: x.replace(" ",""), name_stopword))

['SMITH',
 'JOHNSON',
 'WILLIAMS',
 'JONES',
 'BROWN',
 'DAVIS',
 'MILLER',
 'WILSON',
 'MOORE',
 'TAYLOR',
 'ANDERSON',
 'THOMAS',
 'JACKSON',
 'WHITE',
 'HARRIS',
 'MARTIN',
 'THOMPSON',
 'GARCIA',
 'MARTINEZ',
 'ROBINSON',
 'CLARK',
 'RODRIGUEZ',
 'LEWIS',
 'LEE',
 'WALKER',
 'HALL',
 'ALLEN',
 'YOUNG',
 'HERNANDEZ',
 'KING',
 'WRIGHT',
 'LOPEZ',
 'HILL',
 'SCOTT',
 'GREEN',
 'ADAMS',
 'BAKER',
 'GONZALEZ',
 'NELSON',
 'CARTER',
 'MITCHELL',
 'PEREZ',
 'ROBERTS',
 'TURNER',
 'PHILLIPS',
 'CAMPBELL',
 'PARKER',
 'EVANS',
 'EDWARDS',
 'COLLINS',
 'STEWART',
 'SANCHEZ',
 'MORRIS',
 'ROGERS',
 'REED',
 'COOK',
 'MORGAN',
 'BELL',
 'MURPHY',
 'BAILEY',
 'RIVERA',
 'COOPER',
 'RICHARDSON',
 'COX',
 'HOWARD',
 'WARD',
 'TORRES',
 'PETERSON',
 'GRAY',
 'RAMIREZ',
 'JAMES',
 'WATSON',
 'BROOKS',
 'KELLY',
 'SANDERS',
 'PRICE',
 'BENNETT',
 'WOOD',
 'BARNES',
 'ROSS',
 'HENDERSON',
 'COLEMAN',
 'JENKINS',
 'PERRY',
 'POWELL',
 'LONG',
 'PATTERSON',
 'HUGHES',
 'FLORES',
 'WASHINGTON',
 'BUTLE

In [48]:
stopword = audit_stopword+curr_stopword+datenum_stopword+generic_stopword+genlong_stopword+geo_stopword+name_stopword

# dumping total Stopword and neg/pos words list to a file 

## Enter your own specific path of local disk drive for correct generation of output files

In [63]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\stopword.txt', 'w') as f:
    for i in stopword:
        f.write(i)
        f.write('\n')
    f.close

In [200]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\pos_words.txt', 'w') as f:
    for i in pos_words:
        f.write(i)
        f.write('\n')
    f.close
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\neg_words.txt', 'w') as f:
    for i in neg_words:
        f.write(i)
        f.write('\n')
    f.close

# Reading stopwords from above file to avoid reading them and making a total list again.

In [7]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\stopword.txt', 'r') as f:
    stopword = []
    lst  = f.readlines()
    for l in lst:
        stopword.append(l.replace("\n",""))

In [201]:
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\pos_words.txt', 'r') as f:
    pos_words = []
    lst  = f.readlines()
    for l in lst:
        pos_words.append(l.replace("\n",""))
        
with open(r'Z:\Major Project\assignment\20211030 Test Assignment\New folder\neg_words.txt', 'r') as f:
    neg_words = []
    lst  = f.readlines()
    for l in lst:
        neg_words.append(l.replace("\n",""))

# Analysis :


In [10]:
# changing the case of stopwords to lower for easier analysis
stopword = list(map(lambda x: x.lower(), stopword))

In [11]:
# Change to lowercase
result['Title'] = result['Title'].map(lambda x: x.lower())
result['Body'] = result['Body'].map(lambda x: x.lower())

# Remove numbers
result['Title'] = result['Title'].map(lambda x: re.sub(r'\d+', '', x))
result['Body'] = result['Body'].map(lambda x: re.sub(r'\d+', '', x))

# Remove white spaces
result['Title'] = result['Title'].map(lambda x: x.strip())
result['Body'] = result['Body'].map(lambda x: x.strip())

In [12]:
# Combine title and Body to analyse further
result['Text'] = result['Title'].astype(str) + " " + result['Body']

In [13]:
# Tokenize into words
result['Text'] = result['Text'].map(lambda x: word_tokenize(x))

In [14]:
# Remove non alphabetic tokens
result['Text'] = result['Text'].map(lambda x: [word for word in x if word.isalpha()])

## The analysis listed below will be done before cleaning from stop words.
## Analysis of Readability, complex word count, avg num of words per sentence, syllable count per word, personal pronoun, average word length.

In [15]:
#Syllable count per word and character length analysis according to text analysis document.
lst_syllable = []
lst_char = []
for idx in result.index:
    lst = []
    char_lst = []
    for i in result['Text'][idx]:
        syllable_count=0
        char_count = 0
        for w in i:
            if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                syllable_count=syllable_count+1
            char_count+=1
        if (i[-1:-3:-1] == 'es' or i[-1:-3:-1] == 'ed'):
            syllable_count -= 1
        char_lst.append(char_count)
        lst.append(syllable_count)
    lst_syllable.append(lst)
    lst_char.append(char_lst)

In [16]:
# Starting analyzing the text by storing the character length and syllable count per word in a dataframe: Syll_per_word
x = pd.Series(lst_syllable)
syll_per_word = pd.DataFrame(x, columns = ['Syllable per Word'])

In [17]:
#Complex Word Count analysis
syll_per_word['Complex Word Count'] = syll_per_word['Syllable per Word'].map(lambda x: [1 if syll>2 else 0 for syll in x])
syll_per_word['Complex Word Count'] = syll_per_word['Complex Word Count'].map(lambda x: x.count(1))

In [18]:
# splitting the body of the text into sentences.
result['Sentences'] = result['Body'].map(lambda x: x.split('.'))

In [19]:
# Combining the character length per word to the analysis dataframe.
syll_per_word['char_length_per_word'] = pd.Series(lst_char)

In [20]:
# Avg_word_length
syll_per_word['Avg_word_length'] = syll_per_word['char_length_per_word'].map(lambda x: sum(x)/len(x))

In [21]:
# counting the total number of sentences into analysis dataframe.
syll_per_word['total_num_of_sentences'] = result['Sentences'].map(lambda x: len(x))

In [22]:
# Avg_num_words_per_sentence
syll_per_word['Avg_num_words_per_sentence'] = syll_per_word['char_length_per_word'].map(lambda x: len(x))/syll_per_word['total_num_of_sentences']

In [23]:
# Analysis of Readability
syll_per_word['avg_sentence_length'] = syll_per_word['char_length_per_word'].map(lambda x: len(x))/syll_per_word['total_num_of_sentences']

In [24]:
syll_per_word['percent_complex_words'] = syll_per_word['Complex Word Count']/syll_per_word['char_length_per_word'].map(lambda x: len(x))

In [25]:
syll_per_word['fog_index'] = 0.4*(syll_per_word['avg_sentence_length']+syll_per_word['percent_complex_words'])

In [126]:
# personal pronoun count
lst_personal_pro = [] 
for idx in result.index:
    str1 = result.Body[idx]
    dict1 = {'i':0,'we':0,'my':0,'ours':0,'us':0}
    r = re.compile(r'(?:\bi\b|\bwe\b|\bmy\b|\bours\b|\bus\b)')
    for i in r.findall(str1):
        if i=='we':dict1['we']+=1
        if i=='i':dict1['i']+=1
        if i=='my':dict1['my']+=1
        if i=='ours':dict1['ours']+=1
        if i=='us':dict1['us']+=1
    lst_personal_pro.append(dict1)       

In [129]:
syll_per_word['personal_pronoun'] = pd.Series(lst_personal_pro)

# Cleaning word list from total stopword list and using stopwords package of nltk and performing Sentiment analysis and cleaned Word count analysis.

In [139]:
# Turn lists back to string in order to remove punctuatuion as translate method doesnt work on a list object
result['Text'] = result['Text'].map(lambda x: ' '.join(x))
# Remove Punctuation
result['Text']  = result['Text'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
# Tokenize again in order to filter out stop words from nltk package
result['Text'] = result['Text'].map(lambda x: word_tokenize(x))
# filter out stop words
stop_words = set(stopwords.words('english'))
result['Text'] = result['Text'].map(lambda x: [w for w in x if not w in stop_words])

In [143]:
# cleaning text from total stopword list given in the stopword folder:
result['Text'] = result['Text'].map(lambda x: [w for w in x if not w in stopword])

In [152]:
# Word Count
syll_per_word['word_count'] = result['Text'].map(lambda x: len(x))

In [162]:
# Sentiment Analysis:
syll_per_word['positive_score'] = result['Text'].map(lambda x: sum([1 for w in x if w in pos_words]))

In [168]:
syll_per_word['negative_score'] = result['Text'].map(lambda x: sum([1 for w in x if w in neg_words]))

In [170]:
syll_per_word['polarity_score'] = (syll_per_word['positive_score'] - syll_per_word['negative_score'])/((syll_per_word['positive_score'] + syll_per_word['negative_score'])+0.000001)

In [176]:
syll_per_word['subjectivity_score'] = (syll_per_word['positive_score'] + syll_per_word['negative_score'])/(syll_per_word['word_count']+ 0.000001)

### dumping the analysis dataframe to disk.

In [180]:
syll_per_word.to_csv('unfinal.csv')

### generating proper output file and dumping to disk.

In [187]:
df1 = syll_per_word[['positive_score','negative_score','polarity_score','subjectivity_score','avg_sentence_length','percent_complex_words', 'fog_index', 'Avg_num_words_per_sentence', 'Complex Word Count', 'word_count', 'Syllable per Word', 'personal_pronoun', 'Avg_word_length' ]]
df2 = result[['URL_ID','URL']]

In [193]:
# Combining the analysis dataframe and input dataframe to generate the output dataframe.
unfinal = df2.join(df1)

In [195]:
# renaming the columns for proper representation.
dict = {'positive_score': 'POSITIVE SCORE',
        'negative_score': 'NEGATIVE SCORE',
        'polarity_score': 'POLARITY SCORE',
        'subjectivity_score': 'SUBJECTIVITY SCORE',
        'avg_sentence_length': 'AVG SENTENCE LENGTH',
        'percent_complex_words': 'PERCENTAGE OF COMPLEX WORDS',
        'fog_index': 'FOG INDEX',
        'Avg_num_words_per_sentence': 'AVG NUMBER OF WORDS PER SENTENCE',
        'Complex Word Count': 'COMPLEX WORD COUNT',
        'word_count': 'WORD COUNT',
        'Syllable per Word': 'SYLLABLE PER WORD',
        'personal_pronoun': 'PERSONAL PRONOUNS',
        'Avg_word_length': 'AVG WORD LENGTH'}
 
final = unfinal.rename(columns=dict)

# Dumping the final output to disk for submission.

In [197]:
final.to_excel ('Output Data Structure.xlsx', index = None, header=True)